In [1]:
TESTING_SUBSET = True  # ***** FOR TESTING ONLY !!!!!!!! ******
INFERENCE_ONLY = True

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

    # Enable line-wrapping
    # from IPython.display import HTML, display

    # def set_css():
    #     display(HTML('''
    #         <style>
    #         pre {
    #             white-space: pre-wrap;
    #             }
    #         </style>
    #     '''))
    # get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install transformers
    %pip install datasets
    %pip install pynvml

In [3]:
if IN_COLAB:
    # Enable line-wrapping
    from IPython.display import HTML, display

    def set_css():
        display(HTML('''
            <style>
            pre {
                white-space: pre-wrap;
                }
            </style>
        '''))
    get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from functools import partialmethod
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
from l_utils import *
from model_utils import *
from fe_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

/home/mahesh/Desktop/ML/kaggle/LearningEquality/learning-equality-code/le-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-17 10:22:33.285117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-04-17 10:22:33.423220: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2023-04-17 10:22:33.878717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-17 10:22:33.878810: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-17 10:22:33.878816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [5]:
if IN_COLAB:
    class CFG:
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = '/content/drive/MyDrive/ML/lecr/models/'
else:
    from cfg import *

MiniLM_CFG = {
    "NAME" : "MiniLM",
    "TRANS_NAME" : "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "TRANS_OUT_EMB_DIM" : 384,
    "LR" : 2e-4,
    "BATCH_SIZE" : 800
}
Mpnet_CFG = {
    "NAME" : "mpnet",
    "TRANS_NAME" : "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "TRANS_OUT_EMB_DIM" : 768,
    "LR" : 2e-5,
    "BATCH_SIZE" : 450
}

Xlm_Roberta_CFG = {
    "NAME" : "xlm_roberta",
    "TRANS_NAME" : "xlm-roberta-base",
    "TRANS_OUT_EMB_DIM" : 768,
    "LR" : 2e-6,
    "BATCH_SIZE" : 450
}

MODEL_CFG = MiniLM_CFG

USE_FOCAL_LOSS = False
# Add option to load pre-existing model
LOAD_CHECKPOINT_MODEL = False
if INFERENCE_ONLY:
    LOAD_CHECKPOINT_MODEL = True
if TESTING_SUBSET or INFERENCE_ONLY:
    SAVE_BEST_MODEL = False
else:
    SAVE_BEST_MODEL = True
DRY_RUN = False
TRAIN_PATIENCE = 5
METRIC = 'f2'
METRIC_KEY = None
GAMMA = 2.0
MAX_STRING_LENGTH = 1250

topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_for_ranker.pq"
contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_for_ranker.pq"

TRAIN_CORR_IDS_F        = CFG.DIRNAME_PROCESSED_DATA + "ranker_train_corr_ids.f"
VAL_PIPELINE_CORR_IDS_F = CFG.DIRNAME_PROCESSED_DATA + "ranker_val_pipeline_corr_ids.f"
VAL_CORR_IDS_F          = CFG.DIRNAME_PROCESSED_DATA + "ranker_val_corr_ids.f"

In [6]:
auto_tokenizer = AutoTokenizer.from_pretrained(MODEL_CFG["TRANS_NAME"])

In [7]:
!nvidia-smi

Mon Apr 17 10:22:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   37C    P0    63W / 275W |     31MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# Split data into folds

# Feature engineering ... use same as the one used for bce-lightning

# Training for a Top-k metric ... see if this performs better than directly using minilm embeddings

In [9]:
def progress_bar_control(show_progress_bars = True):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=(not show_progress_bars))

In [10]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
# correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

In [11]:
def limit_str(r):
    return (r[:MAX_STRING_LENGTH]) if pd.notnull(r) else r

#
# Limit the size of description and text in contents
#
contents_df['description'] = contents_df['description'].apply(lambda r: limit_str(r))
contents_df['text'] = contents_df['text'].apply(lambda r: limit_str(r))

In [12]:
# topics_tokenized_df   = pd.read_parquet(topics_tokenized_pq_file)
# contents_tokenized_df = pd.read_parquet(contents_tokenized_pq_file)

In [13]:
pt('Tokenizing data ... ')
topics_tokenized_df, contents_tokenized_df = ranker_tokenize_main(auto_tokenizer, topics_df.copy(),  contents_df.copy(), max_length = 64)

print('')

2023-04-17 10:22:37.955732 : Tokenizing data ... 


Map (num_proc=2):   0% 0/61517 [00:00<?, ? examples/s]

Map (num_proc=2):   2% 1000/61517 [00:00<00:45, 1342.46 examples/s]

Map (num_proc=2):   5% 3000/61517 [00:00<00:14, 3963.74 examples/s]

Map (num_proc=2):   8% 5000/61517 [00:01<00:09, 6039.68 examples/s]

...

Map (num_proc=2):  97% 150000/154047 [00:37<00:00, 4538.03 examples/s]

Map (num_proc=2):  98% 151000/154047 [00:37<00:00, 4111.81 examples/s]

Map (num_proc=2):  99% 152000/154047 [00:37<00:00, 4511.10 examples/s]

Map (num_proc=2):  99% 153024/154047 [00:38<00:00, 3304.71 examples/s]

Map (num_proc=2): 100% 154024/154047 [00:38<00:00, 3324.16 examples/s]

In [14]:
contents_tokenized_df.info()
topics_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  154047 non-null  object
 1   c_0_input_ids       154047 non-null  object
 2   c_1_attention_mask  154047 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61517 entries, 0 to 61516
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  61517 non-null  object
 1   t_0_input_ids       61517 non-null  object
 2   t_1_attention_mask  61517 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [15]:
# read train and val corr data
train_corr_ids_df        = pd.read_feather(TRAIN_CORR_IDS_F)
val_pipeline_corr_ids_df = pd.read_feather(VAL_PIPELINE_CORR_IDS_F)
val_corr_ids_df          = pd.read_feather(VAL_CORR_IDS_F)

In [16]:
# ***** FOR TESTING ONLY !!!!!!!! ******
if TESTING_SUBSET:
    SAMPLE_SIZE = 1000
    train_corr_ids_df        = train_corr_ids_df.sample(n=SAMPLE_SIZE)
    if not INFERENCE_ONLY:
        val_pipeline_corr_ids_df = val_pipeline_corr_ids_df.sample(n=SAMPLE_SIZE)
        val_corr_ids_df          = val_corr_ids_df.sample(n=SAMPLE_SIZE)


In [17]:
train_df = get_ranker_topic_content_merged_data(auto_tokenizer, train_corr_ids_df, topics_tokenized_df, contents_tokenized_df)
val_pipeline_df = get_ranker_topic_content_merged_data(auto_tokenizer, val_pipeline_corr_ids_df, topics_tokenized_df, contents_tokenized_df, keep_ids = True)
val_df = get_ranker_topic_content_merged_data(auto_tokenizer, val_corr_ids_df, topics_tokenized_df, contents_tokenized_df, keep_ids = True)
    

2023-04-17 10:23:28.809005 : Merging data ... 


2023-04-17 10:23:28.858613 : Concatenating input ids ... 
2023-04-17 10:23:28.862733 : Adjusting padding for concatenated input ids ... 
2023-04-17 10:23:28.869687 : Creating new attention mask ... 
2023-04-17 10:23:28.882117 : Merging data ... 
2023-04-17 10:23:28.963292 : Concatenating input ids ... 


2023-04-17 10:23:29.155138 : Adjusting padding for concatenated input ids ... 


2023-04-17 10:23:30.351935 : Creating new attention mask ... 


2023-04-17 10:23:31.190492 : Merging data ... 
2023-04-17 10:23:31.276832 : Concatenating input ids ... 


2023-04-17 10:23:31.489444 : Adjusting padding for concatenated input ids ... 


2023-04-17 10:23:32.825795 : Creating new attention mask ... 


In [18]:
train_df.info()
val_pipeline_df.info()
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   label           1000 non-null   int64 
 1   input_ids       1000 non-null   object
 2   attention_mask  1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 113372 entries, 0 to 113371
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   t_id            113372 non-null  object
 1   c_id            113372 non-null  object
 2   label           113372 non-null  int64 
 3   input_ids       113372 non-null  object
 4   attention_mask  113372 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 118211 entries, 0 to 118210
Data columns (total 5 columns):
 #   Column          Non-Null Count   D

In [19]:
len(train_df['attention_mask'][0])

136

In [20]:
(val_pipeline_df['label'] == 0).sum()

92544

In [21]:
class RankerDS(torch.utils.data.Dataset):
    def __init__(self, df, val = False):
        self.topic_content = torch.tensor(np.array(df[['input_ids', 'attention_mask']].values.tolist()))
        self.label         = torch.tensor(np.array(df[['label']].values.tolist())).float()
        self.val = val
        if (val):
            self.t_id = np.array(df[['t_id']].values.tolist())
            self.c_id = np.array(df[['c_id']].values.tolist())
            
    def __len__(self):
        return self.topic_content.size(dim=0)
    
    def __getitem__(self, item):
        
        topic_content = self.topic_content[item]
        label         = self.label[item]

        if (self.val):
            t_id = self.t_id[item]
            c_id = self.c_id[item]
            return t_id, c_id, topic_content, label
        else:
            return topic_content, label

In [22]:
train_ds = RankerDS(train_df)
val_pipeline_ds   = RankerDS(val_pipeline_df, val = True)
val_ds   = RankerDS(val_df, val = True)

In [23]:
# val_ds[0:2]

In [24]:
@dataclass
class ValDS_collator:
    """
    Handle String data?
    """

    def __call__(self, features):
    # """
    # Input is features as tuples for a given batch
    # output is a tuple with each of the features stacked up as tensors or numpy arrays
    # """
        t_ids = []
        c_ids = []
        topic_content = []
        labels = []

        for i, row in enumerate(features):
            t_ids.append(row[0])
            c_ids.append(row[1])
            topic_content.append(row[2].unsqueeze(0)) # The batch dimension is squeezed out, so we need to add it back
            labels.append(row[3])

        t_ids = np.concatenate(t_ids, axis = 0)
        c_ids = np.concatenate(c_ids, axis = 0)
        topic_content = torch.concat(topic_content, dim = 0)
        labels = torch.concat(labels, dim = 0)
        return t_ids, c_ids, topic_content, labels

In [25]:
# val_loader   = DataLoader(val_ds  , batch_size=100  , collate_fn = ValDS_collator(), shuffle=True, num_workers = 2)

In [26]:
# for k in tqdm(val_loader, desc="Validation"):
#     print(k[2].size())
#     break

In [27]:
val_ds[0:1][2].size()

torch.Size([1, 2, 136])

In [28]:

class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name, out_embs_size):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
        self.classifier = nn.Linear(out_embs_size, 1)
    
    def get_transformer_output(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        pooled_embeddings=mean_pooling_with_token_embs(outputs[0], attention_mask)
        return pooled_embeddings

    def forward(self, features):
        # Grab embeddings from transformer
        embs = self.get_transformer_output(
            features[:,0,:],
            features[:,1,:]
        )
        logits = self.classifier(embs)
        return logits



In [29]:
try: t_model
except NameError: t_model = None
if t_model is not None:
  del t_model
report_gpu()
#BATCH_SIZE = 760 # 650 works for A100, 20 works good for training for TESTING_SUBSET
if TESTING_SUBSET:
    # 20 works good for training for TESTING_SUBSET
    TRAIN_BATCH_SIZE = 20
    VAL_BATCH_SIZE = 2 * TRAIN_BATCH_SIZE
else:
    TRAIN_BATCH_SIZE = MODEL_CFG["BATCH_SIZE"]
    VAL_BATCH_SIZE = 2*TRAIN_BATCH_SIZE
    # 800 works good for minilm
    # 450 works food for mpnet
train_loader = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers = 2)
val_loader   = DataLoader(val_ds  , batch_size=VAL_BATCH_SIZE  , collate_fn = ValDS_collator(), shuffle=True, num_workers = 2)
val_pipeline_loader   = DataLoader(val_pipeline_ds  , batch_size=VAL_BATCH_SIZE  , collate_fn = ValDS_collator(), shuffle=True, num_workers = 2)

GPU:0
no processes are running
GPU:0
no processes are running


In [30]:
type(val_ds[0:2])

tuple

In [31]:
if USE_FOCAL_LOSS:
    LOSS_TYPE = f'focal_loss_{GAMMA}'
else:
    LOSS_TYPE = 'bce'

MODEL_FILENAME_PREFIX = f'One_trans_ranker_{MODEL_CFG["NAME"]}_{LOSS_TYPE}_ml64_batch_size_{TRAIN_BATCH_SIZE}'
SAVE_MODEL_DIR = CFG.SAVE_MODEL_DIR
trans_model = TransModel( MODEL_CFG["TRANS_NAME"], MODEL_CFG["TRANS_OUT_EMB_DIM"])
optimizer = torch.optim.Adam((list(trans_model.parameters())),
                             lr= MODEL_CFG["LR"],
                             # lr= 2e-6,
                             #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
                             )
model_dict = {'trans_model': trans_model, 'optimizer': optimizer}
if LOAD_CHECKPOINT_MODEL:
    MODEL_FILENAME = SAVE_MODEL_DIR + "/from_colab/One_trans_ranker_MiniLM_bce_ml64_batch_size_800_epoch_9_f2__0.636.pth"
    # t_model, optimizer = load_model(t_model, optimizer = None, filename = MODEL_FILENAME)
    # We'll use a fresh optimizer since we are loading ArcFace pre-trained model.
    # optimizer = torch.optim.Adam((list(t_model.parameters())),
    #                           lr=2e-4,
    #                          #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
    #                          )
    # train_scores = []
    # val_scores = []
    # epoch_count = 0
    # MODEL_FILENAME = None
    load_model( model_dict, filename = MODEL_FILENAME)
    train_data_dict = load_dict(SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
    train_scores, val_scores = train_data_dict['train_scores'], train_data_dict['val_scores']
    epoch_count = len(val_scores)
else:
    train_scores = []
    val_scores = []
    epoch_count = 0
    MODEL_FILENAME = None

trans_model = trans_model.to(device)

if USE_FOCAL_LOSS:
    focal_loss = torch.hub.load(
	    'adeelh/pytorch-multi-class-focal-loss',
	    model='focal_loss',
	    # alpha=[.75, .25],
	    gamma=GAMMA,
	    reduction='mean',
	    device=device,
	    dtype=torch.float32,
	    force_reload=False,
        trust_repo=True
    )
    criterion = focal_loss
else:
    criterion = nn.BCEWithLogitsLoss()

In [32]:
if TESTING_SUBSET or INFERENCE_ONLY:
    NUM_EPOCHS = 1
    VAL_DEBUG = 5
else:
    NUM_EPOCHS = 30
    VAL_DEBUG = 0

DISABLE_TQDM = False
if (DISABLE_TQDM):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=DISABLE_TQDM)

scaler = torch.cuda.amp.GradScaler()

pt('Starting Training')
trans_model.train()
#for epoch in tqdm(range(EPOCHS), desc="Epochs"):
for epoch in range(NUM_EPOCHS):
    # We set the below in case it is frozen while doing eval
    pt(f"***** Epoch {epoch_count} *****")
    if not INFERENCE_ONLY:
        train_loss = 0
        #train_acc = 0
        #train_f2_score = 0
        for (topic_content, label) in tqdm(train_loader, desc="Training"):
            if USE_FOCAL_LOSS:
                label = label.type(torch.LongTensor)
            topic_content   = topic_content.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            with torch.autocast(device_type='cuda', dtype=torch.float16):
                logits = trans_model(topic_content)
                if USE_FOCAL_LOSS:
                    logits_1 = torch.sigmoid(logits)
                    logits = torch.concat([1.0-logits_1, logits_1], axis = 1)
                    label = label.squeeze()
                loss = criterion(logits, label)

            # loss.backward()
            # optimizer.step()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
       
            train_loss += loss.item()
        train_loss = round(train_loss/len(train_loader), 5)
        pt(f'Train : | Loss: {train_loss:.5f} ')
        train_scores.append({'loss':train_loss})
    val_bin_acc, val_bin_f1, val_bin_f2, val_recall, val_precision, val_f2 = compute_val_scores_for_binary_classification(trans_model, device, val_loader, threshold = 0.5, debug = VAL_DEBUG)
    val_pipeline_bin_acc, val_pipeline_bin_f1, val_pipeline_bin_f2, val_pipeline_recall, val_pipeline_precision, val_pipeline_f2 = compute_val_scores_for_binary_classification(trans_model, device, val_pipeline_loader, threshold = 0.5, debug = VAL_DEBUG)
    val_dict = {'bin_acc':val_bin_acc, 'bin_f1': val_bin_f1, 'bin_f2': val_bin_f2, 'recall': val_recall, 'precision': val_precision, 'f2': val_f2}
    val_pipeline_dict = {'bin_acc':val_pipeline_bin_acc, 'bin_f1': val_pipeline_bin_f1, 'bin_f2': val_pipeline_bin_f2, 'recall': val_pipeline_recall, 'precision': val_pipeline_precision, 'f2': val_pipeline_f2}
    pt(f'\nValidation:  {val_dict}')
    pt(f'\nValidation pipeline:  {val_pipeline_dict}')
    val_scores.append(val_dict)
    if SAVE_BEST_MODEL:
        MODEL_FILENAME = save_best_model( 
                                        model_dict = model_dict,
                                        old_model_filename = MODEL_FILENAME,
                                        model_filename_prefix = MODEL_FILENAME_PREFIX,
                                        model_path = SAVE_MODEL_DIR,
                                        save_model_fn = save_model,
                                        train_scores = train_scores,
                                        val_scores = val_scores, 
                                        metric = METRIC, metric_key = METRIC_KEY,
                                        dry_run = DRY_RUN
                                        )
        save_model(model_dict, SAVE_MODEL_DIR + f"{MODEL_FILENAME_PREFIX}.last_model.pth")
        save_dict({'train_scores':train_scores, 'val_scores':val_scores}, SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
        if not check_improvement(val_scores, metric = METRIC, metric_key = METRIC_KEY, patience = TRAIN_PATIENCE):
            pt(f'Exiting training since no improvement in the last {TRAIN_PATIENCE} epochs')
            break
    epoch_count += 1
    


2023-04-17 10:23:41.705608 : Starting Training
2023-04-17 10:23:41.706553 : ***** Epoch 1 *****


Validation:   0% 0/2956 [00:00<?, ?it/s]

Validation:   0% 1/2956 [00:00<28:04,  1.75it/s]

Validation:   0% 3/2956 [00:00<09:41,  5.08it/s]

torch.Size([40, 1])
tensor([[ 0.5642],
        [ 1.3038],
        [ 5.2490],
        [-3.4104],
        [-4.0005]], device='cuda:0')
torch.Size([40, 1])
tensor([[0.6374],
        [0.7865],
        [0.9948],
        [0.0320],
        [0.0180]], device='cuda:0')
torch.Size([40, 1])
tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.]], device='cuda:0')
torch.Size([40])
tensor([0., 1., 1., 0., 0.], device='cuda:0')
(40, 1)
(40, 1)
(40, 1)
(40, 1)


...

Validation: 100% 2829/2835 [02:54<00:00, 16.25it/s]

Validation: 100% 2831/2835 [02:54<00:00, 16.10it/s]

Validation: 100% 2833/2835 [02:54<00:00, 16.16it/s]

Validation: 100% 2835/2835 [02:54<00:00, 16.21it/s]

2023-04-17 10:29:41.070738 : 
Validation:  {'bin_acc': 83.0, 'bin_f1': 0.652, 'bin_f2': 0.69, 'recall': 0.698, 'precision': 0.671, 'f2': 0.636}
2023-04-17 10:29:41.070814 : 
Validation pipeline:  {'bin_acc': 86.0, 'bin_f1': 0.682, 'bin_f2': 0.761, 'recall': 0.772, 'precision': 0.682, 'f2': 0.698}


In [33]:
if IN_COLAB:
    from google.colab import runtime
    runtime.unassign()